In [47]:
import pandas as pd
from datetime import datetime

In [48]:
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import chi2

In [49]:
df_full = pd.read_csv('Datafull.csv', encoding = 'utf-8')
df_festivos = pd.read_csv('festivosm.csv', encoding = 'utf-8', sep =';')
df_full.head(5)

,AÑO,CLASE_ACCIDENTE,DISEÑO,FECHA_ACCIDENTE,GRAVEDAD,MES,NUMCOMUNA,BARRIO,HORA,SEMANA,DIA_DEL_AÑO,DIA_DEL_MES,DIA_SEMANA,LONGITUD,ALTITUD,MOMENTO
0,2014,Otro,Tramo de via,2014-08-05,Con heridos,8,11,Los Conquistadores,07:15:00,Semana 32,217,5,Tuesday,-75.589313,6.239787,Mañana
1,2014,Choque,Interseccion,2014-08-05,Con heridos,8,5,Girardot,14:30:00,Semana 32,217,5,Tuesday,-75.570818,6.300041,Tarde
2,2014,Choque,Tramo de via,2014-08-05,Con heridos,8,14,Patio Bonito,15:50:00,Semana 32,217,5,Tuesday,-75.576574,6.208748,Tarde
3,2014,Otro,Tramo de via,2014-08-05,Con heridos,8,8,Villa Hermosa,13:00:00,Semana 32,217,5,Tuesday,-75.551255,6.259058,Tarde
4,2014,Choque,Tramo de via,2014-08-05,Con heridos,8,10,Los Ángeles,19:30:00,Semana 32,217,5,Tuesday,-75.556150,6.249966,Noche


In [50]:
df_festivos['Fecha'] = pd.to_datetime(df_festivos['Fecha'], format="%m/%d/%Y")

In [51]:
df_festivos.head()

,Fecha,Festivo
0,2014-01-01,1
1,2014-01-06,1
2,2014-03-24,1
3,2014-04-17,1
4,2014-04-18,1


In [52]:
df_full = df_full[['AÑO','MES','FECHA_ACCIDENTE','DIA_DEL_MES','DIA_SEMANA','MOMENTO','NUMCOMUNA','CLASE_ACCIDENTE']]

In [53]:
df_full['FECHA_ACCIDENTE'] = pd.to_datetime(df_full['FECHA_ACCIDENTE'], format="%Y/%m/%d")

In [54]:
df_full['festivo'] = df_full['FECHA_ACCIDENTE'].apply(lambda x: 1 if x in df_festivos['Fecha'].unique() else 0)

In [55]:
Merge1 = df_full
Merge1.head()

,AÑO,MES,FECHA_ACCIDENTE,DIA_DEL_MES,DIA_SEMANA,MOMENTO,NUMCOMUNA,CLASE_ACCIDENTE,festivo
0,2014,8,2014-08-05,5,Tuesday,Mañana,11,Otro,0
1,2014,8,2014-08-05,5,Tuesday,Tarde,5,Choque,0
2,2014,8,2014-08-05,5,Tuesday,Tarde,14,Choque,0
3,2014,8,2014-08-05,5,Tuesday,Tarde,8,Otro,0
4,2014,8,2014-08-05,5,Tuesday,Noche,10,Choque,0


In [56]:
Merge1.shape

(249083, 9)

In [57]:
proof = Merge1.groupby(['FECHA_ACCIDENTE','CLASE_ACCIDENTE'])[['CLASE_ACCIDENTE']].count()
proof.rename({proof.columns[0]: 'Total'}, axis = 1, inplace = True)
proof = proof.reset_index()
proof.head(5)

,FECHA_ACCIDENTE,CLASE_ACCIDENTE,Total
0,2014-07-04,Choque,22
1,2014-07-05,Atropello,25
2,2014-07-05,Caída de Ocupante,18
3,2014-07-05,Choque,63
4,2014-07-05,Otro,7


In [58]:
proof['CLASE_ACCIDENTE'].unique()

array(['Choque', 'Atropello', 'Caída de Ocupante', 'Otro', 'Volcamiento',
       'Incendio'], dtype=object)

In [59]:
proof_choque = proof[proof['CLASE_ACCIDENTE'] == 'Choque']
Atropello = proof[proof['CLASE_ACCIDENTE'] == 'Atropello']
Caida = proof[proof['CLASE_ACCIDENTE'] == 'Caída de Ocupante']
Volcamiento = proof[proof['CLASE_ACCIDENTE'] == 'Volcamiento']
Incendio = proof[proof['CLASE_ACCIDENTE'] == 'Incendio']
Otro = proof[proof['CLASE_ACCIDENTE'] == 'Otro']

In [60]:
proof_choque.shape

(2251, 3)

In [61]:
proof_choque.head()

,FECHA_ACCIDENTE,CLASE_ACCIDENTE,Total
0,2014-07-04,Choque,22
3,2014-07-05,Choque,63
8,2014-07-06,Choque,40
12,2014-07-07,Choque,110
17,2014-07-08,Choque,83


In [62]:
Choque = proof_choque[['FECHA_ACCIDENTE','Total']]
Choque_train = Choque[Choque['FECHA_ACCIDENTE'] < datetime(2018, 1, 1)]
Choque_test = Choque[(Choque['FECHA_ACCIDENTE'] >= datetime(2018, 1, 1)) & (Choque['FECHA_ACCIDENTE'] < datetime(2020,1,1))]
Choque_evalua = Choque[Choque['FECHA_ACCIDENTE'] >= datetime(2020,1,1)]

In [63]:
Atropello =  Atropello[['FECHA_ACCIDENTE','Total']]
Atropello_train = Atropello[Atropello['FECHA_ACCIDENTE'] < datetime(2018, 1, 1)]
Atropello_test = Atropello[(Atropello['FECHA_ACCIDENTE'] >= datetime(2018, 1, 1)) & (Atropello['FECHA_ACCIDENTE'] < datetime(2020,1,1))]
Atropello_evalua = Atropello[Atropello['FECHA_ACCIDENTE'] >= datetime(2020,1,1)]

In [64]:
Caida =  Caida[['FECHA_ACCIDENTE','Total']]
Caida_train = Caida[Caida['FECHA_ACCIDENTE'] < datetime(2018, 1, 1)]
Caida_test = Caida[(Caida['FECHA_ACCIDENTE'] >= datetime(2018, 1, 1)) & (Caida['FECHA_ACCIDENTE'] < datetime(2020,1,1))]
Caida_evalua = Caida[Caida['FECHA_ACCIDENTE'] >= datetime(2020,1,1)]

In [65]:
Volcamiento =  Volcamiento[['FECHA_ACCIDENTE','Total']]
Volcamiento_train = Volcamiento[Volcamiento['FECHA_ACCIDENTE'] < datetime(2018, 1, 1)]
Volcamiento_test = Volcamiento[(Volcamiento['FECHA_ACCIDENTE'] >= datetime(2018, 1, 1)) & (Volcamiento['FECHA_ACCIDENTE'] < datetime(2020,1,1))]
Volcamiento_evalua = Volcamiento[Volcamiento['FECHA_ACCIDENTE'] >= datetime(2020,1,1)]

In [66]:
Otro =  Otro[['FECHA_ACCIDENTE','Total']]
Otro_train = Otro[Otro['FECHA_ACCIDENTE'] < datetime(2018, 1, 1)]
Otro_test = Otro[(Otro['FECHA_ACCIDENTE'] >= datetime(2018, 1, 1)) & (Otro['FECHA_ACCIDENTE'] < datetime(2020,1,1))]
Otro_evalua = Otro[Otro['FECHA_ACCIDENTE'] >= datetime(2020,1,1)]

In [67]:
Incendio =  Incendio[['FECHA_ACCIDENTE','Total']]
Incendio_train = Incendio[Incendio['FECHA_ACCIDENTE'] < datetime(2018, 1, 1)]
Incendio_test = Incendio[(Incendio['FECHA_ACCIDENTE'] >= datetime(2018, 1, 1)) & (Incendio['FECHA_ACCIDENTE'] < datetime(2020,1,1))]
Incendio_evalua = Incendio[Incendio['FECHA_ACCIDENTE'] >= datetime(2020,1,1)]

In [70]:
Choque_train.to_csv('Choque_train.csv', encoding = 'utf-8', index = False)
Choque_test.to_csv('Choque_test.csv', encoding = 'utf-8', index = False)
Choque_evalua.to_csv('Choque_evalua.csv', encoding = 'utf-8', index = False)

In [71]:
Atropello_train.to_csv('Atropello_train.csv', encoding = 'utf-8', index = False)
Atropello_test.to_csv('Atropello_test.csv', encoding = 'utf-8', index = False)
Atropello_evalua.to_csv('Atropello_evalua.csv', encoding = 'utf-8', index = False)

In [72]:
Caida_train.to_csv('Caida_train.csv', encoding = 'utf-8', index = False)
Caida_test.to_csv('Caida_test.csv', encoding = 'utf-8', index = False)
Caida_evalua.to_csv('Caida_evalua.csv', encoding = 'utf-8', index = False)

In [73]:
Volcamiento_train.to_csv('Volcamiento_train.csv', encoding = 'utf-8', index = False)
Volcamiento_test.to_csv('Volcamiento_test.csv', encoding = 'utf-8', index = False)
Volcamiento_evalua.to_csv('Volcamiento_evalua.csv', encoding = 'utf-8', index = False)

In [74]:
Otro_train.to_csv('Otro_train.csv', encoding = 'utf-8', index = False)
Otro_test.to_csv('Otro_test.csv', encoding = 'utf-8', index = False)
Otro_evalua.to_csv('Otro_evalua.csv', encoding = 'utf-8', index = False)

In [75]:
Incendio_train.to_csv('Incendio_train.csv', encoding = 'utf-8', index = False)
Incendio_test.to_csv('Incendio_test.csv', encoding = 'utf-8', index = False)
Incendio_evalua.to_csv('Incendio_evalua.csv', encoding = 'utf-8', index = False)

In [77]:
Atropello.head()

,FECHA_ACCIDENTE,Total
1,2014-07-05,25
6,2014-07-06,14
10,2014-07-07,10
15,2014-07-08,7
20,2014-07-09,14
